In [53]:
import pandas  as pd

In [54]:
df = pd.read_csv("Air_Quality.csv", parse_dates=["Start_Date"])
print(f"extracted {len(df)} rows")
df.head()

extracted 18862 rows


,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message
0,336867,375,Nitrogen dioxide (NO2),Mean,ppb,CD,407,Flushing and Whitestone (CD7),Winter 2014-15,2014-12-01,23.97,NaN
1,336741,375,Nitrogen dioxide (NO2),Mean,ppb,CD,107,Upper West Side (CD7),Winter 2014-15,2014-12-01,27.42,NaN
2,550157,375,Nitrogen dioxide (NO2),Mean,ppb,CD,414,Rockaway and Broad Channel (CD14),Annual Average 2017,2017-01-01,12.55,NaN
3,412802,375,Nitrogen dioxide (NO2),Mean,ppb,CD,407,Flushing and Whitestone (CD7),Winter 2015-16,2015-12-01,22.63,NaN
4,412803,375,Nitrogen dioxide (NO2),Mean,ppb,CD,407,Flushing and Whitestone (CD7),Summer 2016,2016-06-01,14.00,NaN


In [55]:
print(df["Start_Date"].mean())
print(df["Start_Date"].min())
print(df["Start_Date"].max())


2015-03-01 00:12:40.385961216
2005-01-01 00:00:00
2023-06-01 00:00:00


In [56]:
# Simulate a last extraction date
with open("last_extraction.txt", "w") as f:
    f.write('2015-03-01 00:12:40.385961216') 

In [57]:
# getting last extraction date from file
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()
last_extraction_date = pd.to_datetime(last_extraction)
print(f"Last extraction date: {last_extraction_date}")
# Filter the DataFrame for rows after the last extraction date
df_filtered = df[df["Start_Date"] > last_extraction_date]
print(f"Filtered {len(df_filtered)} rows after last extraction date")
df_filtered.head()


Last extraction date: 2015-03-01 00:12:40.385961216
Filtered 9120 rows after last extraction date


,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message
2,550157,375,Nitrogen dioxide (NO2),Mean,ppb,CD,414,Rockaway and Broad Channel (CD14),Annual Average 2017,2017-01-01,12.55,NaN
3,412802,375,Nitrogen dioxide (NO2),Mean,ppb,CD,407,Flushing and Whitestone (CD7),Winter 2015-16,2015-12-01,22.63,NaN
4,412803,375,Nitrogen dioxide (NO2),Mean,ppb,CD,407,Flushing and Whitestone (CD7),Summer 2016,2016-06-01,14.00,NaN
5,412676,375,Nitrogen dioxide (NO2),Mean,ppb,CD,107,Upper West Side (CD7),Winter 2015-16,2015-12-01,26.43,NaN
6,412677,375,Nitrogen dioxide (NO2),Mean,ppb,CD,107,Upper West Side (CD7),Summer 2016,2016-06-01,19.10,NaN


In [58]:
#updating the last extraction date
new_extractiondate = df['Start_Date'].max()
with open ("last_extraction.txt",'w') as g:
    g.write(new_extractiondate.isoformat())  # Write the new date in the same format
print(f"New extractiondate written: {new_extractiondate.isoformat()}")


New extractiondate written: 2023-06-01T00:00:00


In [59]:
#creating full and incremental DataFrames
df_full=df.copy()
df_incremental = df[df["Start_Date"] > last_extraction_date].copy()

In [60]:
df_full["Data Value"].max()  # using the Data Value column to check for max value to create a new column

np.float64(424.7)

In [61]:
df_full = df_full.drop_duplicates()
# Convert Data Value from float to int
df_full["Data Value"] = df_full["Data Value"].astype(int)
# Drop 'Measure' and 'Message' column (same value everywhere and message is empty)
df_full = df_full.drop(columns=["Measure","Message"])
# Create a new column for pollution level based on Data Value
df_full["Pollution_Level"] = pd.cut(df_full["Data Value"],
                                    bins=[0, 50, 100, 150, 200, 300],
                                    labels=["Good", "Moderate", "Unhealthy", "Very Unhealthy", "Hazardous"])
df_full.head()
df_full.to_csv("transformed_full.csv", index=False)


In [62]:
# Transforming the incremental DataFrame
# Convert Data Value from float to int
df_incremental["Data Value"] = df_incremental["Data Value"].astype(int)
# Create a new column for pollution level based on Data Value
df_incremental["Pollution_Level"] = pd.cut(df_incremental["Data Value"],
                                            bins=[0, 50, 100, 150, 200, 300],
                                            labels=["Good", "Moderate", "Unhealthy", "Very Unhealthy", "Hazardous"])
# Fill NaN values in 'Message' column with "None"
df_incremental["Message"] = df_incremental["Message"].fillna("None")

# Drop 'Measure' and 'Message' column(same value everywhere))
df_incremental = df_incremental.drop(columns=["Measure"])

new_extractiondate = df["Start_Date"].max()
with open("last_extraction.txt", "w") as g:
    g.write(new_extractiondate.isoformat())
    
# Save the transformed DataFrames to CSV files
df_incremental.to_csv("transformed_incremental.csv", index=False)
